In [2]:
import os 
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder 
from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import ssl 
import pymysql
import os
from sklearn.externals import joblib

# used to fix Python SSL CERTIFICATE_VERIFY_FAILED
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
#Load data as we need to do min-max normalization
project_data_path = '/Users/charles/Desktop/Project/LongTan_File/'
longtan_file_2018 = os.path.join(project_data_path, 'LongTan_complete_2018.csv')
print('Path of read in data: %s' % (longtan_file_2018))

Path of read in data: /Users/charles/Desktop/Project/LongTan_File/LongTan_complete_2018.csv


In [4]:
project_data = pd.read_csv(longtan_file_2018)

In [5]:
#Restore model
clf=joblib.load("./LT_ML_Predict.pkl")

In [6]:
project_data.head()

,station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,LT,2018,1,1,0,14.0,0.25,0.4,2.4,46.0,58.0,19.0,0.0,75.0,1.0,71.0,71.0,7.1,6.7
1,LT,2018,1,1,1,14.1,0.25,0.8,1.7,49.0,58.0,23.0,0.0,75.0,1.4,69.0,72.0,6.9,6.5
2,LT,2018,1,1,2,14.2,0.26,0.7,1.7,50.0,46.0,27.0,0.0,74.0,1.3,67.0,67.0,7.3,6.2
3,LT,2018,1,1,3,14.3,0.25,0.7,1.6,49.0,38.0,24.0,0.0,74.0,1.4,68.0,64.0,6.3,6.1
4,LT,2018,1,1,4,14.2,0.25,0.6,1.9,48.0,37.0,23.0,0.0,74.0,1.5,61.0,58.0,5.0,5.2


In [7]:
project_data.tail()

,station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
8755,LT,2018,12,31,19,14.5,0.24,1.5,9.4,34.0,10.0,13.0,0.0,95.0,2.3,58.0,57.0,7.0,6.4
8756,LT,2018,12,31,20,14.5,0.24,1.4,9.2,34.0,11.0,17.0,0.0,95.0,2.2,53.0,50.0,5.6,5.9
8757,LT,2018,12,31,21,14.6,0.25,1.5,9.1,33.0,8.0,13.0,0.0,95.0,1.9,52.0,54.0,5.7,5.5
8758,LT,2018,12,31,22,14.6,0.22,1.2,7.7,34.0,6.0,4.0,0.0,95.0,2.1,49.0,48.0,6.1,5.6
8759,LT,2018,12,31,23,14.7,0.19,1.0,5.8,36.0,6.0,1.0,0.0,96.0,2.0,48.0,52.0,6.3,6.3


In [8]:
#Connect MySQL
db = pymysql.connect(host='localhost', port=3306, user='root', db='Air_pollution', charset='utf8')
cursor = db.cursor()

cursor.execute("SELECT Station,Year,Month,Day,Time,SO2,CO,O3,PM10,NOx,NO,NO2,WIND_SPEED,WIND_DIREC,AMB_TEMP,RAINFALL,RH,WS_HR,WD_HR,PM25 FROM LongTan_real_time_2020 ORDER BY ID DESC LIMIT 1")
#cursor.execute("select * from LongTan_real_time_2020 ORDER BY ID DESC LIMIT 1")

#Fetch data
LongTan_real_time_2020 = cursor.fetchone()

#Close connection
db.close()

print(LongTan_real_time_2020)

('LongTan', 2020, 1, 6, 15, 1.6, 0.49, 51, 52, 17.0, 3.0, 14.0, 2.2, 267, 21.6, 0, 2, 74.0, 250, 38)


In [9]:
#Show type of LongTan_real_time_2020
type(LongTan_real_time_2020)

tuple

In [10]:
#Convert tuple to list
LongTan_real_time_2020 = list(LongTan_real_time_2020)

In [11]:
#Show MySQL column name
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]

In [12]:
type(field_names)

list

In [13]:
#Convert list to dictionary
#Convert dictionary to dataframe
list_dict = zip(field_names, LongTan_real_time_2020)
dict_dataframe = dict(list_dict)
df_complete = pd.DataFrame([dict_dataframe ])

In [14]:
df_complete

,AMB_TEMP,CO,Day,Month,NO,NO2,NOx,O3,PM10,PM25,RAINFALL,RH,SO2,Station,Time,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR,Year
0,21.6,0.49,6,1,3.0,14.0,17.0,51,52,38,0,2,1.6,LongTan,15,250,267,2.2,74.0,2020


In [15]:
df_complete = df_complete[['Station',
 'Year',
 'Month',
 'Day',
 'Time',
 'AMB_TEMP',
 'CO',
 'NO',
 'NOx',
 'O3',
 'PM10',
 'PM25',
 'RAINFALL',
 'RH',
 'SO2',
 'WD_HR',
 'WIND_DIREC',
 'WIND_SPEED',
 'WS_HR']]

In [16]:
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM25,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,LongTan,2020,1,6,15,21.6,0.49,3.0,17.0,51,52,38,0,2,1.6,250,267,2.2,74.0


In [17]:
df_complete = df_complete.rename(columns={"PM25": "PM2.5"})
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,LongTan,2020,1,6,15,21.6,0.49,3.0,17.0,51,52,38,0,2,1.6,250,267,2.2,74.0


In [18]:
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(0,22.5),1, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(22.5,67.5),2, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(67.5,112.5),3, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(112.5,157.5),4, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(157.5,202.5),5, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(202.5,247.5),6, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(247.5,292.5),7, df_complete['WD_HR'])
df_complete['WD_HR'] = np.where(df_complete['WD_HR'].between(292.5,337.5),8, df_complete['WD_HR'])
df_complete['WD_HR'] = df_complete['WD_HR'].mask(df_complete['WD_HR'] > 337.5, 1)

In [19]:
#Convert WIND_DIREC to 8 degree
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(0,22.5),1, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(22.5,67.5),2, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(67.5,112.5),3, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(112.5,157.5),4, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(157.5,202.5),5, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(202.5,247.5),6, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(247.5,292.5),7, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = np.where(df_complete['WIND_DIREC'].between(292.5,337.5),8, df_complete['WIND_DIREC'])
df_complete['WIND_DIREC'] = df_complete['WIND_DIREC'].mask(df_complete['WIND_DIREC'] > 337.5, 1)

In [20]:
#Convert WIND_SPEED to 4 degree
df_complete['WIND_SPEED'] = np.where(df_complete['WIND_SPEED'].between(0,0.3),1, df_complete['WIND_SPEED'])
df_complete['WIND_SPEED'] = np.where(df_complete['WIND_SPEED'].between(0.3,3.4),2, df_complete['WIND_SPEED'])
df_complete['WIND_SPEED'] = np.where(df_complete['WIND_SPEED'].between(3.4,8),3, df_complete['WIND_SPEED'])
df_complete['WIND_SPEED'] = df_complete['WIND_SPEED'].mask(df_complete['WIND_SPEED'] > 8, 4)

In [21]:
#Convert WS_HR to 4 degree
df_complete['WS_HR'] = np.where(df_complete['WS_HR'].between(0,0.3),1, df_complete['WS_HR'])
df_complete['WS_HR'] = np.where(df_complete['WS_HR'].between(0.3,3.4),2, df_complete['WS_HR'])
df_complete['WS_HR'] = np.where(df_complete['WS_HR'].between(3.4,8),3, df_complete['WS_HR'])
df_complete['WS_HR'] = df_complete['WS_HR'].mask(df_complete['WS_HR'] > 8, 4)

In [22]:
#Convert type to integer because of one hot encoding 
df_complete['WS_HR']= df_complete['WS_HR'].round(0).astype(int)
df_complete['WD_HR']= df_complete['WD_HR'].round(0).astype(int)
df_complete['WIND_DIREC']= df_complete['WIND_DIREC'].round(0).astype(int)
df_complete['WIND_SPEED']= df_complete['WIND_SPEED'].round(0).astype(int)

In [23]:
#One hot encoding
df_complete_WD_HR_OHE = pd.get_dummies(df_complete['WD_HR'],prefix=['WD_HR_OHE'])
df_complete_WIND_DIREC_OHE = pd.get_dummies(df_complete['WIND_DIREC'],prefix=['WIND_DIREC_OHE'])
df_complete_WIND_SPEED_OHE = pd.get_dummies(df_complete['WIND_SPEED'],prefix=['WIND_SPEED_OHE'])
df_complete_WS_HR_OHE = pd.get_dummies(df_complete['WS_HR'],prefix=['WS_HR_OHE'])

In [24]:
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,LongTan,2020,1,6,15,21.6,0.49,3.0,17.0,51,52,38,0,2,1.6,7,7,2,4


In [25]:
#Create a list and insert column name 
create_OHE_WD_HR_column_name = ("['WD_HR_OHE']_1",
 "['WD_HR_OHE']_2",
 "['WD_HR_OHE']_3",
 "['WD_HR_OHE']_4",
 "['WD_HR_OHE']_5",
 "['WD_HR_OHE']_6",
 "['WD_HR_OHE']_7",
 "['WD_HR_OHE']_8",
)

In [26]:
#Convert type
create_OHE_WD_HR_column_name = list(create_OHE_WD_HR_column_name)

In [27]:
#Create zero*7 list and return a list with 8 values
def display(value):
    s1 = [0]*7 
    s1.insert(value-1,1)
    return s1

In [28]:
#Fetch value to insert function
WD_HR_OHE_list_value = display(df_complete.iloc[0,15])

In [29]:
#Convert list to dictionary
#Convert dictionary to dataframe
OHE_WD_HR_list_dict = zip(create_OHE_WD_HR_column_name , WD_HR_OHE_list_value)
OHE_WD_HR_dict_dataframe = dict(OHE_WD_HR_list_dict)
OHE_WD_HR_complete = pd.DataFrame([OHE_WD_HR_dict_dataframe])

In [30]:
#Show dataframe
OHE_WD_HR_complete

,['WD_HR_OHE']_1,['WD_HR_OHE']_2,['WD_HR_OHE']_3,['WD_HR_OHE']_4,['WD_HR_OHE']_5,['WD_HR_OHE']_6,['WD_HR_OHE']_7,['WD_HR_OHE']_8
0,0,0,0,0,0,0,1,0


In [31]:
#Create a list and insert column name 
create_OHE_WIND_DIREC_column_name= ("['WIND_DIREC_OHE']_1",
 "['WIND_DIREC_OHE']_2",
 "['WIND_DIREC_OHE']_3",
 "['WIND_DIREC_OHE']_4",
 "['WIND_DIREC_OHE']_5",
 "['WIND_DIREC_OHE']_6",
 "['WIND_DIREC_OHE']_7",
 "['WIND_DIREC_OHE']_8",
)

In [32]:
#Convert type
create_OHE_WIND_DIREC_column_name = list(create_OHE_WIND_DIREC_column_name)

In [33]:
#Fetch value to insert function
WIND_DIREC_OHE_list_value = display(df_complete.iloc[0,16])

In [34]:
#Convert list to dictionary
#Convert dictionary to dataframe
OHE_WIND_DIREC_list_dict = zip(create_OHE_WIND_DIREC_column_name , WIND_DIREC_OHE_list_value)
OHE_WIND_DIREC_dict_dataframe = dict(OHE_WIND_DIREC_list_dict)
OHE_WIND_DIREC_complete = pd.DataFrame([OHE_WIND_DIREC_dict_dataframe])

In [35]:
#Show dataframe
OHE_WIND_DIREC_complete 

,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,['WIND_DIREC_OHE']_3,['WIND_DIREC_OHE']_4,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8
0,0,0,0,0,0,0,1,0


In [36]:
#Create a list and insert column name 
create_OHE_WIND_SPEED_column_name = ("['WIND_SPEED_OHE']_2",
 "['WIND_SPEED_OHE']_3",
 "['WIND_SPEED_OHE']_4",
)

In [37]:
#Convert type
create_OHE_WIND_SPEED_column_name = list(create_OHE_WIND_SPEED_column_name)

In [38]:
#As never have 1 at ['WIND_SPEED_OHE']_1 in history
def display_(value):
    if value != 1:
        s1 = [0]*2 
        s1.insert(value-2,1)
    else:
        s1 = [0]*3
    return s1

In [39]:
#Fetch value to insert function
WIND_SPEED_OHE_list_value = display_(df_complete.iloc[0,17])

In [40]:
#Convert list to dictionary
#Convert dictionary to dataframe
OHE_WIND_SPEED_list_dict = zip(create_OHE_WIND_SPEED_column_name , WIND_SPEED_OHE_list_value)
OHE_WIND_SPEED_dict_dataframe = dict(OHE_WIND_SPEED_list_dict)
OHE_WIND_SPEED_complete = pd.DataFrame([OHE_WIND_SPEED_dict_dataframe])

In [41]:
OHE_WIND_SPEED_complete 

,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4
0,1,0,0


In [42]:
#Create a list and insert column name 
create_OHE_WS_HR_column_name = ("['WS_HR_OHE']_2",
 "['WS_HR_OHE']_3",
 "['WS_HR_OHE']_4"
)

In [43]:
#Convert type
create_OHE_WS_HR_column_name = list(create_OHE_WS_HR_column_name)

In [44]:
#Fetch value to insert function
WS_HR_OHE_list_value = display_(df_complete.iloc[0,18])

In [45]:
#Convert list to dictionary
#Convert dictionary to dataframe
OHE_WS_HR_list_dict = zip(create_OHE_WS_HR_column_name , WS_HR_OHE_list_value)
OHE_WS_HR_dict_dataframe = dict(OHE_WS_HR_list_dict)
OHE_WS_HR_complete = pd.DataFrame([OHE_WS_HR_dict_dataframe ])

In [46]:
OHE_WS_HR_complete

,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,0,0,1


In [47]:
#Concatenate dataframe after doing one hot encoding in each component
Wind_OHE_df_complete = pd.concat([OHE_WD_HR_complete,OHE_WIND_DIREC_complete,OHE_WIND_SPEED_complete,OHE_WS_HR_complete], axis=1)
Wind_OHE_df_complete 

,['WD_HR_OHE']_1,['WD_HR_OHE']_2,['WD_HR_OHE']_3,['WD_HR_OHE']_4,['WD_HR_OHE']_5,['WD_HR_OHE']_6,['WD_HR_OHE']_7,['WD_HR_OHE']_8,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,...,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [48]:
df_complete

,Station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,LongTan,2020,1,6,15,21.6,0.49,3.0,17.0,51,52,38,0,2,1.6,7,7,2,4


In [49]:
df_complete = df_complete.iloc[:,5:15]
df_complete

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2
0,21.6,0.49,3.0,17.0,51,52,38,0,2,1.6


In [50]:
#Select columns to insert model
new_x_dataframe= pd.concat([df_complete,Wind_OHE_df_complete], axis=1)
new_x_dataframe

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,...,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,21.6,0.49,3.0,17.0,51,52,38,0,2,1.6,...,0,0,1,0,1,0,0,0,0,1


In [51]:
#Min-Max Normalization
new_x_dataframe['AMB_TEMP'] = (new_x_dataframe['AMB_TEMP']-project_data['AMB_TEMP'].min())/(project_data['AMB_TEMP'].max()-project_data['AMB_TEMP'].min())
new_x_dataframe['CO'] = (new_x_dataframe['CO']-project_data['CO'].min())/(project_data['CO'].max()-project_data['CO'].min())
new_x_dataframe['NO'] = (new_x_dataframe['NO']-project_data['NO'].min())/(project_data['NO'].max()-project_data['NO'].min())
new_x_dataframe['NOx'] = (new_x_dataframe['NOx']-project_data['NOx'].min())/(project_data['NOx'].max()-project_data['NOx'].min())
new_x_dataframe['O3'] = (new_x_dataframe['O3']-project_data['O3'].min())/(project_data['O3'].max()-project_data['O3'].min())
new_x_dataframe['PM10'] = (new_x_dataframe['PM10']-project_data['PM10'].min())/(project_data['PM10'].max()-project_data['PM10'].min())
new_x_dataframe['RAINFALL'] = (new_x_dataframe['RAINFALL']-project_data['RAINFALL'].min())/(project_data['RAINFALL'].max()-project_data['RAINFALL'].min())
new_x_dataframe['RH'] = (new_x_dataframe['RH']-project_data['RH'].min())/(project_data['RH'].max()-project_data['RH'].min())
new_x_dataframe['SO2'] = (new_x_dataframe['SO2']-project_data['SO2'].min())/(project_data['SO2'].max()-project_data['SO2'].min())
new_x_dataframe['PM2.5'] = (new_x_dataframe['PM2.5']-project_data['PM2.5'].min())/(project_data['PM2.5'].max()-project_data['PM2.5'].min())


In [52]:
new_x_dataframe

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,...,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,0.467877,0.627374,0.325798,0.428799,0.572152,0.554876,0.679799,0.0,-0.551686,0.288169,...,0,0,1,0,1,0,0,0,0,1


In [53]:
#Convert predict_y to list
predict_y = clf.predict(new_x_dataframe.values).tolist()

In [54]:
#Convert list to string
predict_y = ' '.join(map(str,predict_y)) 

In [55]:
#Show final prediction
print(predict_y) 

17.82933580315536


In [58]:
predict_y_redis = round(float(predict_y),2)
predict_y_redis 

17.83

In [73]:
import redis
conn = redis.StrictRedis(host="localhost", port=6379, charset="utf-8", decode_responses=True)
conn.hset('prediction','LongTan',predict_y_redis)
print(conn.hgetall('prediction'))

{'GuanYin': '25.83', 'TaoYuan': '21.09', 'PingZhen': '19.02', 'JhongLi': '27.77', 'LongTan': '17.83'}
